In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup

# 셀레니움 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)
driver.get("https://www.koicd.kr/ins/act.do")
time.sleep(2)

data = []

for page in range(1, 101):  # 💡 원하면 더 늘릴 수 있음
    print(f"📄 수집 중: {page}페이지")
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = soup.select("table.tableList tr")[1:]  # 헤더 제외

    if not rows:
        print("📭 더 이상 데이터 없음. 종료")
        break

    for row in rows:
        tds = row.find_all("td")
        if len(tds) >= 3:
            suga_code = tds[0].get_text(strip=True)
            act_kor = tds[1].get_text(strip=True)
            act_eng = tds[2].get_text(strip=True)
            data.append({
                "suga_code": suga_code,
                "act_kor_name": act_kor,
                "act_eng_name": act_eng
            })

    # 다음 페이지 이동
    driver.execute_script(f"pageMove({page + 1})")
    time.sleep(1.5)

driver.quit()

# CSV 저장
df = pd.DataFrame(data)
df.to_csv("suga_info_all.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: suga_info_all.csv")


📄 수집 중: 1페이지
📭 더 이상 데이터 없음. 종료
✅ 저장 완료: suga_info_all.csv


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time, pandas as pd

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.koicd.kr/ins/act.do")
time.sleep(2)

data = []
for page in range(1, 101):
    print(f"📄 수집 중: {page}페이지")
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = soup.select("table.tableList tr")[1:]
    if not rows:
        print("📭 끝났습니다.")
        break
    for row in rows:
        tds = row.find_all("td")
        if len(tds) >= 3:
            data.append({
                "suga_code": tds[0].get_text(strip=True),
                "act_kor_name": tds[1].get_text(strip=True),
                "act_eng_name": tds[2].get_text(strip=True)
            })
    driver.execute_script(f"pageMove({page+1})")
    time.sleep(1.5)

driver.quit()
df = pd.DataFrame(data)
df.info()
df.to_csv("suga_info_all.csv", index=False, encoding="utf-8-sig")
print("✅ 완료: suga_info_all.csv")


📄 수집 중: 1페이지
📭 끝났습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
✅ 완료: suga_info_all.csv


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

driver = webdriver.Chrome()  # 크롬드라이버 경로 명시 안 하면 PATH에 있어야 작동
driver.get("https://www.koicd.kr/ins/act.do")  # 또는 웹주소

time.sleep(2)  # JS 로딩 대기

# 👉 iframe이 있다면 아래 코드 추가
# driver.switch_to.frame(driver.find_element(By.TAG_NAME, "iframe"))

# 👉 table 나타날 때까지 기다렸다가 크롤링
rows = driver.find_elements(By.CSS_SELECTOR, "table.tableList tr")

data = []
for row in rows[1:]:  # 헤더 제외
    cols = row.find_elements(By.TAG_NAME, "td")
    if cols:
        data.append([col.text.strip() for col in cols])

df = pd.DataFrame(data, columns=["수가코드", "항목명", "버튼 등등"])  # 컬럼은 적절히 수정
df.to_csv("suga_info_all.csv", index=False, encoding="utf-8-sig")
driver.quit()


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

# 크롬 옵션 설정
options = Options()
options.add_argument("--headless")  # 창 안 띄우고 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# ⬇ ChromeDriver 경로가 PATH에 없다면 명시해줘야 함
driver = webdriver.Chrome(options=options)

# ⬇ 형님이 직접 내려받은 파일이라면 file:// 경로로 열고
html_path = "https://www.koicd.kr/ins/act.do"  # 또는 웹 주소
driver.get(html_path)

time.sleep(2)  # JS 렌더링 기다리기 (필요시 더 늘려도 됨)

# 👉 table 로딩 대기 후 행 가져오기
rows = driver.find_elements(By.CSS_SELECTOR, "table.tableList > tbody > tr")

data = []
for row in rows[1:]:  # 첫 줄은 헤더일 수 있으니 skip
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text.strip() for col in cols])

driver.quit()

# 👉 DataFrame으로 저장
df = pd.DataFrame(data)
df.columns = ["진료행위코드", "명칭", "기준구분", "코드시작일", "코드종료일", "삭제여부", "상세"][:len(df.columns)]
df.to_csv("suga_info_all.csv", index=False, encoding="utf-8-sig")
print("✅ 완료: suga_info_all.csv")


✅ 완료: suga_info_all.csv


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 🧠 크롬 설정
options = Options()
options.add_argument('--headless')  # 창 없이
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

url = "https://www.koicd.kr/ins/act.do"
driver.get(url)

time.sleep(3)  # JS 로딩 대기

all_data = []

# ▶ 1~5페이지만 예시로 (전체 534까지는 비효율적이므로 페이지네이션 개선 필요)
for page in range(1, 6):
    print(f"🔄 {page}페이지 진입")
    # 페이지 이동
    page_btn = wait.until(EC.element_to_be_clickable((By.XPATH, f"//a[text()='{page}']")))
    driver.execute_script("arguments[0].click();", page_btn)
    time.sleep(2)

    # 행마다 루프
    rows = driver.find_elements(By.CSS_SELECTOR, "table.tableList tbody tr")[1:]
    for row in rows:
        try:
            code = row.find_elements(By.TAG_NAME, "td")[0].text.strip()
            name = row.find_elements(By.TAG_NAME, "td")[1].text.strip()
            detail_btn = row.find_element(By.CSS_SELECTOR, "button.btnTableView")
            driver.execute_script("arguments[0].click();", detail_btn)
            time.sleep(1)

            # 상세창 정보
            detail_box = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "detailAct")))
            detail_texts = detail_box.text.split("\n")
            record = {
                "수가코드": code,
                "행위명(한글)": name
            }
            for line in detail_texts:
                if "분류코드" in line:
                    record["분류코드"] = line.split(":")[1].strip()
                if "행위명(영문)" in line:
                    record["행위명(영문)"] = line.split(":")[1].strip()
                if "치과병의원단가" in line:
                    record["치과병의원단가"] = line.split(":")[1].strip()
                if "보건기관단가" in line:
                    record["보건기관단가"] = line.split(":")[1].strip()
            all_data.append(record)

            # 닫기
            close_btn = driver.find_element(By.XPATH, "//button[text()='닫기']")
            driver.execute_script("arguments[0].click();", close_btn)
            time.sleep(0.5)
        except Exception as e:
            print(f"❌ Error: {e}")
            continue

driver.quit()

df = pd.DataFrame(all_data)
df.to_csv("suga_info_koicd_sample.csv", index=False, encoding="utf-8-sig")
print("✅ 완료: suga_info_koicd_sample.csv")


🔄 1페이지 진입
🔄 2페이지 진입
🔄 3페이지 진입
🔄 4페이지 진입
🔄 5페이지 진입
✅ 완료: suga_info_koicd_sample.csv


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# ✅ 크롬드라이버 경로 수정 필요
chrome_driver_path = r"C:\tools\chromedriver-win64\chromedriver.exe"

options = Options()
# options.add_argument('--headless')  # 필요시 주석처리
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)
driver.get("https://www.koicd.kr/ins/act.do")

wait = WebDriverWait(driver, 10)
time.sleep(2)

results = []

# ✅ 5페이지만 테스트 (534 페이지 있음)
for page_num in range(1, 6):
    print(f"▶ {page_num}페이지 진입")
    
    # 테이블 로딩 대기
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr")))
    time.sleep(1)

    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 3:
                continue

            suga_code = tds[0].text.strip()
            han_name = tds[1].text.strip()
            eng_name = tds[2].text.strip()

            # 상세 버튼 클릭 (돋보기)
            row.find_element(By.CSS_SELECTOR, ".fa-search-plus").click()
            time.sleep(1)

            # 오른쪽 패널 내용 가져오기
            info_box = driver.find_element(By.CLASS_NAME, "suga-view")
            infos = info_box.find_elements(By.TAG_NAME, "tr")

            detail_dict = {
                "수가코드": suga_code,
                "행위명(한글)": han_name,
                "행위명(영문)": eng_name,
            }

            for tr in infos:
                try:
                    th = tr.find_element(By.TAG_NAME, "th").text.strip()
                    td = tr.find_element(By.TAG_NAME, "td").text.strip()
                    detail_dict[th] = td
                except:
                    continue

            results.append(detail_dict)

            # 닫기
            driver.find_element(By.CLASS_NAME, "btn-close").click()
            time.sleep(0.5)
        except Exception as e:
            print("⚠️ 오류:", e)
            continue

    # 다음 페이지 클릭
    try:
        next_btn = driver.find_element(By.LINK_TEXT, str(page_num + 1))
        next_btn.click()
        time.sleep(2)
    except:
        print("📌 마지막 페이지 도달")
        break

driver.quit()

# ✅ 저장
df = pd.DataFrame(results)
df.to_csv("suga_info_koicd_final.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: suga_info_koicd_final.csv")


▶ 1페이지 진입
⚠️ 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".fa-search-plus"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff78ac16f95+76917]
	GetHandleVerifier [0x0x7ff78ac16ff0+77008]
	(No symbol) [0x0x7ff78a9c9dea]
	(No symbol) [0x0x7ff78aa20256]
	(No symbol) [0x0x7ff78aa2050c]
	(No symbol) [0x0x7ff78aa12d4c]
	(No symbol) [0x0x7ff78aa484af]
	(No symbol) [0x0x7ff78aa12c16]
	(No symbol) [0x0x7ff78aa48680]
	(No symbol) [0x0x7ff78aa7065c]
	(No symbol) [0x0x7ff78aa48243]
	(No symbol) [0x0x7ff78aa11431]
	(No symbol) [0x0x7ff78aa121c3]
	GetHandleVerifier [0x0x7ff78aeed2cd+3051437]
	GetHandleVerifier [0x0x7ff78aee7923+3028483]
	GetHandleVerifier [0x0x7ff78af058bd+3151261]
	GetHandleVerifier [0x0x7ff78ac3185e+185662]
	GetHandleVerifier [0x0x7ff78ac3971f+218111]
	GetHandleVer

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def crawl_suga_codes():
    driver = init_driver()
    driver.get("https://www.koicd.kr/ins/act.do")

    # ✅ 대기: 테이블 로딩될 때까지 기다림
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#resultList tr"))
        )
    except:
        print("❌ 첫 로딩 실패")
        driver.quit()
        return

    suga_data = []
    page = 1

    while True:
        print(f"▶ {page}페이지 진입")
        time.sleep(1)

        rows = driver.find_elements(By.CSS_SELECTOR, "#resultList tr")
        print(f"   ↪️ rows 탐색 결과: {len(rows)}개")

        for row in rows:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 4:
                continue

            code = tds[0].text.strip()
            name_ko = tds[1].text.strip()
            name_en = tds[2].text.strip()

            try:
                # ✅ 버튼 직접 클릭
                detail_btn = tds[3].find_element(By.CLASS_NAME, "btn-info")
                driver.execute_script("arguments[0].click();", detail_btn)

                WebDriverWait(driver, 5).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "#actViewLayer .panel-body"))
                )

                detail_text = driver.find_element(By.CSS_SELECTOR, "#actViewLayer .panel-body").text.strip()

                suga_data.append({
                    "code": code,
                    "name_ko": name_ko,
                    "name_en": name_en,
                    "detail": detail_text
                })

                close_btn = driver.find_element(By.CSS_SELECTOR, "#actViewLayer .btn-default")
                driver.execute_script("arguments[0].click();", close_btn)

            except Exception as e:
                print(f"⚠️ 상세 정보 실패: {code}, 오류: {e}")
                continue

        # ✅ 다음 버튼 여부
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, ".pagination li.active + li a")
            if "javascript:void(0)" in next_btn.get_attribute("href"):
                print("▶ 마지막 페이지 도달.")
                break
            else:
                page += 1
                next_btn.click()
        except:
            print("▶ 다음 버튼 없음.")
            break

    driver.quit()

    # 저장
    import csv
    with open("suga_info.csv", "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["code", "name_ko", "name_en", "detail"])
        writer.writeheader()
        writer.writerows(suga_data)

    print(f"✅ 총 {len(suga_data)}건 저장 완료 (suga_info.csv)")


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def crawl_suga_codes():
    driver = init_driver()
    driver.get("https://www.koicd.kr/ins/act.do")

    # ✅ 대기: 테이블 로딩될 때까지 기다림
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#resultList tr"))
        )
    except:
        print("❌ 첫 로딩 실패")
        driver.quit()
        return

    suga_data = []
    page = 1

    while True:
        print(f"▶ {page}페이지 진입")
        time.sleep(1)

        rows = driver.find_elements(By.CSS_SELECTOR, "#resultList tr")
        print(f"   ↪️ rows 탐색 결과: {len(rows)}개")

        for row in rows:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 4:
                continue

            code = tds[0].text.strip()
            name_ko = tds[1].text.strip()
            name_en = tds[2].text.strip()

            try:
                # ✅ 버튼 직접 클릭
                detail_btn = tds[3].find_element(By.CLASS_NAME, "btn-info")
                driver.execute_script("arguments[0].click();", detail_btn)

                WebDriverWait(driver, 5).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "#actViewLayer .panel-body"))
                )

                detail_text = driver.find_element(By.CSS_SELECTOR, "#actViewLayer .panel-body").text.strip()

                suga_data.append({
                    "code": code,
                    "name_ko": name_ko,
                    "name_en": name_en,
                    "detail": detail_text
                })

                close_btn = driver.find_element(By.CSS_SELECTOR, "#actViewLayer .btn-default")
                driver.execute_script("arguments[0].click();", close_btn)

            except Exception as e:
                print(f"⚠️ 상세 정보 실패: {code}, 오류: {e}")
                continue

        # ✅ 다음 버튼 여부
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, ".pagination li.active + li a")
            if "javascript:void(0)" in next_btn.get_attribute("href"):
                print("▶ 마지막 페이지 도달.")
                break
            else:
                page += 1
                next_btn.click()
        except:
            print("▶ 다음 버튼 없음.")
            break

    driver.quit()

    # 저장
    import csv
    with open("suga_info.csv", "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["code", "name_ko", "name_en", "detail"])
        writer.writeheader()
        writer.writerows(suga_data)

    print(f"✅ 총 {len(suga_data)}건 저장 완료 (suga_info.csv)")


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# Chrome 설정
options = Options()
# options.add_argument('--headless')  # GUI 없이 실행하고 싶을 경우
driver = webdriver.Chrome(options=options)

try:
    print("🔄 접속 시도 중...")
    driver.get("https://www.koicd.kr/ins/act.do")
    time.sleep(3)  # 💡 JS 로딩 대기 (필요시 WebDriverWait으로 개선 가능)

    # ✅ 핵심: class가 'act_table'인 테이블 가져오기
    table = driver.find_element(By.CLASS_NAME, "act_table")

    # 🧪 디버깅 출력 (예시: HTML 일부 보기)
    print("✅ 테이블 일부:\n", table.get_attribute("outerHTML")[:500])

except Exception as e:
    print(f"❌ 실패: {e}")

finally:
    driver.quit()


🔄 접속 시도 중...
✅ 테이블 일부:
 <table class="act_table"><thead><tr><th style="width:5%;"></th><th style="width:10%;">수가코드</th><th style="width:40%;">행위명(한글)</th><th style="width:40%;">행위명(영문)</th><th style="width:5%;"></th></tr></thead><tbody><tr class="5323" data-seq="5323"><td class="child"><b value="AA100">+</b></td><td class="insfee" title=""><b class="find"></b>A<b class="find"></b>A<b class="find"></b>1<b class="find"></b>0<b class="find"></b>0<b class="find"></b></td><td class="name" title="초진진찰료-치과의원, 보건의료원 내 치과"><b c


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 드라이버 초기화 (형님 이미 하셨을 테니 생략 가능)
driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
wait = WebDriverWait(driver, 10)

# 1️⃣ 수가코드 테이블에서 + 버튼들 찾기
expand_buttons = driver.find_elements(By.XPATH, "//td[@class='child']/b")

for i, btn in enumerate(expand_buttons):
    try:
        # 2️⃣ 상위 코드 추출
        code = btn.get_attribute("value")
        print(f"▶ 상위코드: {code}")

        # 3️⃣ 클릭해서 하위 테이블 펼치기
        driver.execute_script("arguments[0].click();", btn)

        # 4️⃣ 하위 테이블 로딩 대기
        wait.until(
            EC.presence_of_element_located(
                (By.XPATH, f"//tr[@class='child-detail']//table[contains(@class, 'act_detail_table')]")
            )
        )
        time.sleep(0.3)  # 혹시 몰라 딜레이

        # 5️⃣ 하위 테이블 찾기
        detail_table = driver.find_element(By.XPATH, f"//tr[@class='child-detail']//table[contains(@class, 'act_detail_table')]")
        rows = detail_table.find_elements(By.TAG_NAME, "tr")

        print(f"  ▶ 하위 항목 수: {len(rows)-1}")  # 첫 줄은 헤더
        for row in rows[1:]:
            cols = row.find_elements(By.TAG_NAME, "td")
            values = [col.text.strip() for col in cols]
            print(f"    - {' | '.join(values)}")

    except Exception as e:
        print(f"❌ [{i}] 코드 {code} 수집 실패: {e}")
        continue

driver.quit()


In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 1️⃣ 브라우저 열기 (화면 보이게)
options = Options()
# options.add_argument('--headless')  # 형님 디버깅 시 꺼두세요
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

print("🔄 접속 시도 중...")
driver.get("https://www.koicd.kr/ins/act.do")

# 2️⃣ 테이블 로딩 대기
try:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.act_table")))
    print("✅ 수가코드 테이블 로딩 완료")
except:
    print("❌ 테이블 로딩 실패")
    driver.quit()
    exit()

# 3️⃣ 첫 번째 + 버튼 클릭 시도
try:
    plus_btn = driver.find_element(By.XPATH, "(//td[@class='child']/b)[1]")
    code = plus_btn.get_attribute("value")
    print(f"▶ 첫 수가코드: {code}")
    driver.execute_script("arguments[0].click();", plus_btn)
    time.sleep(1)  # JS 로딩 기다림

    # 4️⃣ 하위 테이블 로딩 대기
    wait.until(
        EC.presence_of_element_located(
            (By.XPATH, "//tr[@class='child-detail']//table[contains(@class, 'act_detail_table')]")
        )
    )

    # 5️⃣ 하위 항목 테이블 찾기
    detail_table = driver.find_element(By.XPATH, "//tr[@class='child-detail']//table[contains(@class, 'act_detail_table')]")
    rows = detail_table.find_elements(By.TAG_NAME, "tr")

    print(f"  ▶ 하위 항목 수: {len(rows)-1}")
    for row in rows[1:]:  # 헤더 제외
        cols = row.find_elements(By.TAG_NAME, "td")
        values = [col.text.strip() for col in cols]
        print("    -", " | ".join(values))

except Exception as e:
    print(f"❌ 하위 항목 수집 중 오류 발생: {e}")

finally:
    driver.quit()


🔄 접속 시도 중...
✅ 수가코드 테이블 로딩 완료
▶ 첫 수가코드: AA100
❌ 하위 항목 수집 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff774d0e935+77845]
	GetHandleVerifier [0x0x7ff774d0e990+77936]
	(No symbol) [0x0x7ff774ac9cda]
	(No symbol) [0x0x7ff774b206aa]
	(No symbol) [0x0x7ff774b2095c]
	(No symbol) [0x0x7ff774b73d07]
	(No symbol) [0x0x7ff774b4890f]
	(No symbol) [0x0x7ff774b70b07]
	(No symbol) [0x0x7ff774b486a3]
	(No symbol) [0x0x7ff774b11791]
	(No symbol) [0x0x7ff774b12523]
	GetHandleVerifier [0x0x7ff774fe684d+3059501]
	GetHandleVerifier [0x0x7ff774fe0c0d+3035885]
	GetHandleVerifier [0x0x7ff775000400+3164896]
	GetHandleVerifier [0x0x7ff774d28c3e+185118]
	GetHandleVerifier [0x0x7ff774d3054f+216111]
	GetHandleVerifier [0x0x7ff774d172e4+113092]
	GetHandleVerifier [0x0x7ff774d17499+113529]
	GetHandleVerifier [0x0x7ff774cfe298+10616]
	BaseThreadInitThunk [0x0x7ffa8fcae8d7+23]
	RtlUserThreadStart [0x0x7ffa90f5c34c+44]



In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
time.sleep(2)

# 1. 테이블 안에서 수가코드 1개만 클릭
table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")

for row in rows:
    cls = row.get_attribute("class")
    if cls.isdigit():  # 상위 항목만 추출
        try:
            toggle_btn = row.find_element(By.CSS_SELECTOR, "td.child b")
            toggle_btn.click()  # 하위 펼침
            print(f"✅ {cls} 클릭 완료")
            time.sleep(1)

            # 하위 tr 확인
            sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.childAA100.table")
            print(f"▶ 하위 항목 {len(sub_rows)}개 수집됨")
        except Exception as e:
            print(f"❌ {cls} 처리 중 오류:", e)
        break


✅ 5323 클릭 완료
▶ 하위 항목 49개 수집됨


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
wait = WebDriverWait(driver, 10)
time.sleep(2)

all_data = []

# 페이지 수
MAX_PAGE = 534

for page in range(1, MAX_PAGE + 1):
    print(f"▶ 페이지 {page} 접속 중...")
    try:
        # 테이블 로딩 대기
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.act_table")))
        table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
        rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")

        # for row in rows:
        #     cls = row.get_attribute("class")
        #     if cls and cls.isdigit():
        #         try:
        #             # 상위 toggle 클릭
        #             toggle_btn = row.find_element(By.CSS_SELECTOR, "td.child b")
        #             toggle_btn.click()
        #             time.sleep(0.3)

        #             # 하위 항목 추출
        #             sub_rows = table.find_elements(By.CSS_SELECTOR, "tr.childAA100.table[style='display: table-row;']")
        #             for sub_row in sub_rows:
        #                 tds = sub_row.find_elements(By.TAG_NAME, "td")
        #                 row_data = [td.text.strip() for td in tds]
        #                 if any(row_data):  # 내용 있는 행만
        #                     all_data.append(row_data)

        #         except Exception as e:
        #             print(f"❌ 상위 tr 처리 중 오류: {e}")
        
        for row in rows:
            cls = row.get_attribute("class")
            if cls and cls.isdigit():
                try:
                    suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                    toggle_btn = row.find_element(By.CSS_SELECTOR, "td.child b")
                    driver.execute_script("arguments[0].click();", toggle_btn)  # JS 클릭

                    print(f"✅ {cls} ({suga_code}) 클릭 완료")
                    time.sleep(1)

                    sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
                    print(f"▶ {suga_code} 하위 항목 {len(sub_rows)}개 수집됨")

                except Exception as e:
                    print(f"❌ {cls} 처리 중 오류:", e)
                break

        # 다음 페이지 클릭
        next_btn = driver.find_element(By.CSS_SELECTOR, "a[onclick*='goPage']")
        if 'disabled' in next_btn.get_attribute("class"):
            print("마지막 페이지 도달.")
            break
        else:
            driver.execute_script("goPage({});".format(page + 1))
            time.sleep(2)

    except Exception as e:
        print(f"❌ 페이지 {page} 처리 중 오류: {e}")
        break

# 저장
df = pd.DataFrame(all_data)
df.to_csv("suga_info_all.csv", index=False, encoding="utf-8-sig")
print(f"✅ 총 {len(all_data)}건 저장 완료 (suga_info_all.csv)")

driver.quit()


▶ 페이지 1 접속 중...
몇 행? : <built-in method count of list object at 0x00000270BBEE2600>
✅ 5323 (AA100) 클릭 완료
▶ AA100 하위 항목 49개 수집됨
❌ 페이지 1 처리 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[onclick*='goPage']"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff774d0e935+77845]
	GetHandleVerifier [0x0x7ff774d0e990+77936]
	(No symbol) [0x0x7ff774ac9cda]
	(No symbol) [0x0x7ff774b206aa]
	(No symbol) [0x0x7ff774b2095c]
	(No symbol) [0x0x7ff774b73d07]
	(No symbol) [0x0x7ff774b4890f]
	(No symbol) [0x0x7ff774b70b07]
	(No symbol) [0x0x7ff774b486a3]
	(No symbol) [0x0x7ff774b11791]
	(No symbol) [0x0x7ff774b12523]
	GetHandleVerifier [0x0x7ff774fe684d+3059501]
	GetHandleVerifier [0x0x7ff774fe0c0d+3035885]
	GetHandleVerifier [0x0x7ff775000400+3164896]
	GetHandleVerifier [0x0x7ff774d28c3

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
time.sleep(2)

table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")

for row in rows:
    cls = row.get_attribute("class")
    if cls and cls.isdigit():
        try:
            suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
            toggle_btn = row.find_element(By.CSS_SELECTOR, "td.child b")
            driver.execute_script("arguments[0].click();", toggle_btn)
            print(f"✅ {cls} ({suga_code}) 클릭 완료")
            time.sleep(1)

            # 하위 항목 수집 (display 조건 없이 class 기준만)
            sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
            print(f"▶ {suga_code} 하위 항목 {len(sub_rows)}개 수집됨")

        except Exception as e:
            print(f"❌ {cls} ({suga_code}) 처리 중 오류:", e)

        # 💡 rows 다시 불러와야 구조 깨지지 않음
        rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")


✅ 5323 (AA100) 클릭 완료
▶ AA100 하위 항목 49개 수집됨
✅ 5373 (AA106) 클릭 완료
▶ AA106 하위 항목 32개 수집됨
✅ 5406 (AA107) 클릭 완료
▶ AA107 하위 항목 32개 수집됨
✅ 5439 (AA109) 클릭 완료
▶ AA109 하위 항목 33개 수집됨
✅ 5473 (AA154) 클릭 완료
▶ AA154 하위 항목 40개 수집됨
✅ 5514 (AA155) 클릭 완료
▶ AA155 하위 항목 30개 수집됨
✅ 5545 (AA156) 클릭 완료
▶ AA156 하위 항목 32개 수집됨
✅ 5578 (AA157) 클릭 완료
▶ AA157 하위 항목 32개 수집됨
✅ 5611 (AA200) 클릭 완료
▶ AA200 하위 항목 52개 수집됨
✅ 5664 (AA206) 클릭 완료
▶ AA206 하위 항목 34개 수집됨
✅ 5699 (AA207) 클릭 완료
▶ AA207 하위 항목 34개 수집됨
✅ 5734 (AA209) 클릭 완료
▶ AA209 하위 항목 35개 수집됨
❌ 5770 (AA222) 처리 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.child b"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff774d0e935+77845]
	GetHandleVerifier [0x0x7ff774d0e990+77936]
	(No symbol) [0x0x7ff774ac9cda]
	(No symbol) [0x0x7ff774b206aa]
	(No symbol

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
wait = WebDriverWait(driver, 5)

fail_list = []

page = 1
while True:
    print(f"\n▶ 페이지 {page} 접속 중...")
    try:
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.act_table")))
        rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")

        for row in rows:
            try:
                cls = row.get_attribute("class")
                # tr class가 숫자로 시작하는 상위 항목만 대상
                if not cls or not cls.isdigit():
                    continue

                suga_code_el = row.find_element(By.CSS_SELECTOR, "td.insfee")
                suga_code = suga_code_el.text.strip()

                toggle_btns = row.find_elements(By.CSS_SELECTOR, "td.child b")
                if toggle_btns:
                    toggle_btns[0].click()
                    print(f"✅ {cls} ({suga_code}) 클릭 완료")
                    time.sleep(0.7)
                else:
                    print(f"⚠️  {cls} ({suga_code}) 펼침 버튼 없음 - 단일 항목 처리")

                # 하위 항목들 확인
                sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
                print(f"▶ {suga_code} 하위 항목 {len(sub_rows)}개 수집됨")

            except Exception as e:
                print(f"❌ {cls} 처리 중 오류: {e}")
                fail_list.append(cls)

        # 다음 페이지 이동
        next_page = driver.find_elements(By.CSS_SELECTOR, f"a[onclick*='goPage({page + 1}']")  # ex) goPage(2)
        if next_page:
            next_page[0].click()
            page += 1
            time.sleep(1)
        else:
            print("⛔ 다음 페이지 없음. 종료.")
            break

    except Exception as e:
        print(f"❌ 페이지 {page} 처리 중 오류: {e}")
        break

# 실패 항목 저장
if fail_list:
    with open("fail_list.txt", "w", encoding="utf-8") as f:
        for item in fail_list:
            f.write(f"{item}\n")
    print(f"⚠️ 실패 항목 {len(fail_list)}건 저장 완료 (fail_list.txt)")
else:
    print("✅ 모든 항목 정상 처리 완료.")

driver.quit()



▶ 페이지 1 접속 중...
⛔ 다음 페이지 없음. 종료.
✅ 모든 항목 정상 처리 완료.


In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time

driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
time.sleep(2)

all_data = []
failed_codes = []

def extract_row_data(row, parent_code=""):
    try:
        code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
        name_kr = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
        name_en = row.find_element(By.CSS_SELECTOR, "td:nth-child(4)").text.strip()
        all_data.append({
            "수가코드": code,
            "행위명(한글)": name_kr,
            "행위명(영문)": name_en,
            "상위코드": parent_code
        })
    except Exception as e:
        print(f"❌ 항목 파싱 오류: {e}")

def collect_page_data():
    table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
    rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")

    for idx, row in enumerate(rows):
        cls = row.get_attribute("class")
        try:
            # 수가코드 확인용
            suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
        except:
            continue

        # 버튼 없는 단일 항목
        if "child" not in row.get_attribute("innerHTML"):
            extract_row_data(row)
            continue

        try:
            # 펼침 버튼 클릭
            btn = row.find_element(By.CSS_SELECTOR, "td.child b")
            driver.execute_script("arguments[0].click();", btn)
            print(f"✅ {suga_code} 클릭 완료")
            time.sleep(1)

            extract_row_data(row)  # 상위항목도 포함
            # 하위 항목 수집
            sub_rows = driver.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
            print(f"▶ {suga_code} 하위 항목 {len(sub_rows)}개 수집됨")
            for sub in sub_rows:
                extract_row_data(sub, parent_code=suga_code)

        except Exception as e:
            print(f"❌ {suga_code} 처리 실패:", e)
            failed_codes.append(suga_code)

# 페이지 순회
page = 1
while True:
    print(f"▶ 페이지 {page} 접속 중...")
    collect_page_data()

    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a[onclick*='goPage']")
        if ">" not in next_btn.text:
            break
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        page += 1
    except:
        print("🛑 다음 페이지 없음: 종료.")
        break

# 저장
with open("suga_info_all.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["수가코드", "행위명(한글)", "행위명(영문)", "상위코드"])
    writer.writeheader()
    writer.writerows(all_data)

if failed_codes:
    with open("failed_codes.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(failed_codes))
    print(f"⚠ 일부 항목 클릭 실패 ({len(failed_codes)}건): failed_codes.txt 저장 완료")

print("✅ 모든 항목 정상 처리 완료.")
driver.quit()


▶ 페이지 1 접속 중...
✅ AA100 클릭 완료
▶ AA100 하위 항목 49개 수집됨
✅ AA106 클릭 완료
▶ AA106 하위 항목 32개 수집됨
✅ AA107 클릭 완료
▶ AA107 하위 항목 32개 수집됨
✅ AA109 클릭 완료
▶ AA109 하위 항목 33개 수집됨
✅ AA154 클릭 완료
▶ AA154 하위 항목 40개 수집됨
✅ AA155 클릭 완료
▶ AA155 하위 항목 30개 수집됨
✅ AA156 클릭 완료
▶ AA156 하위 항목 32개 수집됨
✅ AA157 클릭 완료
▶ AA157 하위 항목 32개 수집됨
✅ AA200 클릭 완료
▶ AA200 하위 항목 52개 수집됨
✅ AA206 클릭 완료
▶ AA206 하위 항목 34개 수집됨
✅ AA207 클릭 완료
▶ AA207 하위 항목 34개 수집됨
✅ AA209 클릭 완료
▶ AA209 하위 항목 35개 수집됨
❌ AA222 처리 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.child b"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff774d0e935+77845]
	GetHandleVerifier [0x0x7ff774d0e990+77936]
	(No symbol) [0x0x7ff774ac9cda]
	(No symbol) [0x0x7ff774b206aa]
	(No symbol) [0x0x7ff774b2095c]
	(No symbol) [0x0x7ff774b130ac]
	(No symbol) [0x0x7ff774

In [34]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

# ✅ 기본 설정
BASE_URL = "https://www.koicd.kr/ins/act.do"
RESULT_DIR = "./results"
os.makedirs(RESULT_DIR, exist_ok=True)

# ✅ 크롬 드라이버 초기화
driver = webdriver.Chrome()
driver.get(BASE_URL)
time.sleep(2)

# ✅ 결과 저장용 리스트
all_data = []
failed_codes = []

page_num = 1
while True:
    print(f"\n▶ 페이지 {page_num} 접속 중...")

    try:
        # 테이블 찾기
        table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
        rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")

        page_data = []

        for row in rows:
            try:
                cls = row.get_attribute("class")
                data_seq = row.get_attribute("data-seq")
                if not data_seq:
                    continue

                # 상위코드
                suga_code_el = row.find_element(By.CSS_SELECTOR, "td.child b")
                suga_code = suga_code_el.text.strip()
                suga_name_ko = row.find_element(By.CSS_SELECTOR, "td.name[title]").get_attribute("title").strip()

                try:
                    suga_code_el.click()
                    time.sleep(0.5)
                    sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
                    if not sub_rows:
                        raise Exception("하위 없음")
                    print(f"✅ {data_seq} ({suga_code}) 클릭 완료 | 하위 {len(sub_rows)}개")

                    for sub in sub_rows:
                        tds = sub.find_elements(By.TAG_NAME, "td")
                        if len(tds) >= 2:
                            sub_code = tds[0].text.strip()
                            sub_name_ko = tds[1].text.strip()
                            page_data.append({
                                "상위코드": suga_code,
                                "상위명칭": suga_name_ko,
                                "세부코드": sub_code,
                                "세부명칭": sub_name_ko
                            })
                except Exception as e:
                    print(f"⚠️  {data_seq} ({suga_code}) 하위 없음 또는 클릭 실패: {str(e)}")
                    # 하위가 없어도 상위 저장
                    page_data.append({
                        "상위코드": suga_code,
                        "상위명칭": suga_name_ko,
                        "세부코드": "",
                        "세부명칭": ""
                    })
            except (NoSuchElementException, ElementClickInterceptedException) as e:
                try:
                    # 버튼 없는 경우: 상위만 저장
                    suga_code = row.find_element(By.CSS_SELECTOR, "td.name").text.strip()
                    suga_name_ko = row.find_element(By.CSS_SELECTOR, "td.name[title]").get_attribute("title").strip()
                    print(f"🟡 {data_seq} ({suga_code}) 버튼 없음 → 단독 항목 저장")
                    page_data.append({
                        "상위코드": suga_code,
                        "상위명칭": suga_name_ko,
                        "세부코드": "",
                        "세부명칭": ""
                    })
                except Exception as e2:
                    print(f"❌ {data_seq} 처리 실패 → {e2}")
                    failed_codes.append(suga_code if 'suga_code' in locals() else f"SEQ:{data_seq}")
                    continue

        # 중간 저장
        df = pd.DataFrame(page_data)
        csv_path = os.path.join(RESULT_DIR, f"page_{page_num:03}.csv")
        json_path = os.path.join(RESULT_DIR, f"page_{page_num:03}.json")
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        df.to_json(json_path, orient="records", force_ascii=False)
        all_data.extend(page_data)

        # 다음 페이지 버튼 찾기
        next_btns = driver.find_elements(By.CSS_SELECTOR, "a[onclick*='goPage']")
        found = False
        for btn in next_btns:
            if btn.text.strip() == str(page_num + 1):
                btn.click()
                time.sleep(1)
                page_num += 1
                found = True
                break

        if not found:
            print("⛔ 다음 페이지 없음. 종료.")
            break

    except Exception as e:
        print(f"❌ 페이지 {page_num} 처리 중 오류: {e}")
        break

# ✅ 최종 병합 저장
final_df = pd.DataFrame(all_data)
final_df.to_csv(os.path.join(RESULT_DIR, "suga_info_all.csv"), index=False, encoding="utf-8-sig")
final_df.to_json(os.path.join(RESULT_DIR, "suga_info_all.json"), orient="records", force_ascii=False)

# ✅ 실패 항목 저장
with open(os.path.join(RESULT_DIR, "failed_codes.txt"), "w", encoding="utf-8") as f:
    f.writelines([f"{c}\n" for c in failed_codes])

print("✅ 모든 항목 정상 처리 완료.")



▶ 페이지 1 접속 중...
⚠️  5323 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5373 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5406 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5439 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5473 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5514 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5545 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5578 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5611 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5664 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5699 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5734 (+) 하위 없음 또는 클릭 실패: 하위 없음
🟡 5770 (재진-물리치료,주사등을 시술받은 경우) 버튼 없음 → 단독 항목 저장
⚠️  5771 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5808 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5852 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5886 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5921 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5958 (+) 하위 없음 또는 클릭 실패: 하위 없음
⚠️  5973 (+) 하위 없음 또는 클릭 실패: 하위 없음
⛔ 다음 페이지 없음. 종료.
✅ 모든 항목 정상 처리 완료.


In [35]:
import os
import csv
import json
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *

# 저장 경로 설정
os.makedirs("data/csv", exist_ok=True)
os.makedirs("data/json", exist_ok=True)
FAILED_PATH = "failed_codes.txt"
with open(FAILED_PATH, "w") as f:
    pass  # 초기화

# 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.koicd.kr/ins/act.do")
wait = WebDriverWait(driver, 10)

def save_page_data(page_no, records):
    # CSV 저장
    with open(f"data/csv/page_{page_no}.csv", "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["코드", "행위명(한글)", "상위코드"])
        writer.writerows(records)

    # JSON 저장
    with open(f"data/json/page_{page_no}.json", "w", encoding="utf-8") as f:
        json.dump([
            {"code": row[0], "name": row[1], "parent": row[2]} for row in records
        ], f, ensure_ascii=False, indent=2)

def extract_table_rows():
    table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
    return table.find_elements(By.CSS_SELECTOR, "tbody > tr")

def process_page(page_no):
    print(f"\n▶ 페이지 {page_no} 접속 중...")
    time.sleep(1)

    records = []
    rows = extract_table_rows()

    for row in rows:
        cls = row.get_attribute("class")
        if not cls.isdigit():
            continue

        try:
            # 수가코드, 명칭
            suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2) b").text.strip()
            suga_name = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()

            try:
                # 버튼 있는 경우
                btn = row.find_element(By.CSS_SELECTOR, "td.child b")
                driver.execute_script("arguments[0].scrollIntoView(true);", btn)
                wait.until(EC.element_to_be_clickable(btn)).click()
                time.sleep(0.5)

                table = driver.find_element(By.CSS_SELECTOR, "table.act_table")
                sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")

                if sub_rows:
                    for sub in sub_rows:
                        code = sub.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                        name = sub.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
                        records.append((code, name, suga_code))
                    print(f"🟢 {suga_code} 하위 항목 {len(sub_rows)}개 수집됨")
                else:
                    raise NoSuchElementException("하위 없음")

            except NoSuchElementException:
                # 단독항목인 경우
                print(f"🟡 {suga_code} (단독항목) → 단독 저장")
                records.append((suga_code, suga_name, ""))

            except Exception as e:
                print(f"⚠️  {suga_code} (+) 클릭 실패 → {e.__class__.__name__}")
                with open(FAILED_PATH, "a") as f:
                    f.write(suga_code + "\n")

        except Exception as e:
            print(f"❌ {cls} 항목 처리 중 예외: {e.__class__.__name__}")
            continue

    save_page_data(page_no, records)

def go_to_page(num):
    try:
        selector = f"a[onclick*='goPage({num}']"
        btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
        driver.execute_script("arguments[0].scrollIntoView(true);", btn)
        btn.click()
        time.sleep(1)
        return True
    except:
        return False

# 전체 페이지 수 수동 지정 (스크립트 개선 여지 있음)
TOTAL_PAGES = 27
for page in range(1, TOTAL_PAGES + 1):
    if page != 1:
        if not go_to_page(page):
            print(f"⛔ 페이지 {page} 이동 실패. 중단.")
            break
    process_page(page)

print("\n📦 전체 데이터 통합 중...")

# CSV 통합 저장
merged_csv = []
for file in sorted(os.listdir("data/csv")):
    if file.endswith(".csv"):
        with open(f"data/csv/{file}", encoding="utf-8-sig") as f:
            reader = csv.reader(f)
            next(reader)
            merged_csv.extend(reader)

with open("suga_info_all.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["코드", "행위명(한글)", "상위코드"])
    writer.writerows(merged_csv)

# JSON 통합 저장
merged_json = []
for file in sorted(os.listdir("data/json")):
    if file.endswith(".json"):
        with open(f"data/json/{file}", encoding="utf-8") as f:
            merged_json.extend(json.load(f))

with open("suga_info_all.json", "w", encoding="utf-8") as f:
    json.dump(merged_json, f, ensure_ascii=False, indent=2)

print("✅ 모든 항목 정상 처리 완료.")
driver.quit()



▶ 페이지 1 접속 중...
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
🟡  (단독항목) → 단독 저장
⛔ 페이지 2 이동 실패. 중단.

📦 전체 데이터 통합 중...
✅ 모든 항목 정상 처리 완료.


In [36]:
import os
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 기본 설정
BASE_URL = "https://www.koicd.kr/ins/act.do"
OUTPUT_DIR = "suga_results"
JSON_DIR = os.path.join(OUTPUT_DIR, "json_pages")
FAILED_FILE = os.path.join(OUTPUT_DIR, "failed_codes.txt")
CSV_FILE = os.path.join(OUTPUT_DIR, "suga_info_all.csv")

# 폴더 생성
os.makedirs(JSON_DIR, exist_ok=True)

# 드라이버 로드
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get(BASE_URL)
time.sleep(2)

all_data = []
failed_codes = []

page_num = 1
while True:
    print(f"\n▶ 페이지 {page_num} 접속 중...")

    try:
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.act_table")))
        rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")
    except Exception as e:
        print(f"❌ 테이블 로딩 실패: {e}")
        break

    page_data = []

    for row in rows:
        cls = row.get_attribute("class")
        if not cls or not cls.isdigit():
            continue

        try:
            suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
            kor_name = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
            eng_name = row.find_element(By.CSS_SELECTOR, "td:nth-child(4)").text.strip()
            try:
                toggle_btn = row.find_element(By.CSS_SELECTOR, "td.child b")
                driver.execute_script("arguments[0].click();", toggle_btn)
                time.sleep(0.5)

                sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
                sub_data = []
                for sub in sub_rows:
                    tds = sub.find_elements(By.TAG_NAME, "td")
                    if len(tds) >= 3:
                        sub_code = tds[1].text.strip()
                        sub_name = tds[2].text.strip()
                        sub_data.append({
                            "parent_code": suga_code,
                            "sub_code": sub_code,
                            "sub_name": sub_name
                        })

                if sub_data:
                    page_data.extend(sub_data)
                else:
                    print(f"⚠️  {cls} ({suga_code}) 클릭했지만 하위 없음")
                    page_data.append({
                        "parent_code": suga_code,
                        "sub_code": "",
                        "sub_name": kor_name
                    })

            except NoSuchElementException:
                print(f"🟡  {suga_code} (단독항목) → 단독 저장")
                page_data.append({
                    "parent_code": suga_code,
                    "sub_code": "",
                    "sub_name": kor_name
                })

        except Exception as e:
            print(f"❌ {cls} 처리 실패 → {e}")
            try:
                suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                failed_codes.append(suga_code)
            except:
                continue

    # 페이지별 JSON 저장
    json_path = os.path.join(JSON_DIR, f"page_{page_num}.json")
    with open(json_path, "w", encoding="utf-8") as jf:
        json.dump(page_data, jf, ensure_ascii=False, indent=2)

    all_data.extend(page_data)

    # 다음 페이지 이동 시도
    try:
        next_btn = driver.find_element(By.LINK_TEXT, str(page_num + 1))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
        time.sleep(0.5)
        next_btn.click()
        time.sleep(1)
        page_num += 1
    except Exception as e:
        print(f"⛔ 페이지 {page_num+1} 이동 실패. 중단.")
        break

# 최종 CSV 저장
with open(CSV_FILE, "w", newline="", encoding="utf-8-sig") as cf:
    writer = csv.writer(cf)
    writer.writerow(["parent_code", "sub_code", "sub_name"])
    for row in all_data:
        writer.writerow([row["parent_code"], row["sub_code"], row["sub_name"]])

# 실패 로그 저장
with open(FAILED_FILE, "w", encoding="utf-8") as ff:
    for code in failed_codes:
        ff.write(code + "\n")

print("\n📦 전체 데이터 통합 완료.")
print("✅ 모든 항목 정상 처리 완료.")

driver.quit()



▶ 페이지 1 접속 중...
🟡  AA222 (단독항목) → 단독 저장

▶ 페이지 2 접속 중...

▶ 페이지 3 접속 중...

▶ 페이지 4 접속 중...

▶ 페이지 5 접속 중...

▶ 페이지 6 접속 중...

▶ 페이지 7 접속 중...
🟡  AC001 (단독항목) → 단독 저장
🟡  AC002 (단독항목) → 단독 저장
🟡  AC003 (단독항목) → 단독 저장
🟡  AC004 (단독항목) → 단독 저장
🟡  AC005 (단독항목) → 단독 저장
🟡  AC006 (단독항목) → 단독 저장
🟡  AC007 (단독항목) → 단독 저장
🟡  AC031 (단독항목) → 단독 저장
🟡  AC032 (단독항목) → 단독 저장
🟡  AC033 (단독항목) → 단독 저장

▶ 페이지 8 접속 중...
🟡  AC034 (단독항목) → 단독 저장
🟡  AC410 (단독항목) → 단독 저장
🟡  AC421 (단독항목) → 단독 저장
🟡  AC422 (단독항목) → 단독 저장
🟡  AC430 (단독항목) → 단독 저장
🟡  AC600 (단독항목) → 단독 저장
🟡  AC711 (단독항목) → 단독 저장
🟡  AC712 (단독항목) → 단독 저장
🟡  AC721 (단독항목) → 단독 저장
🟡  AC722 (단독항목) → 단독 저장
🟡  AC731 (단독항목) → 단독 저장

▶ 페이지 9 접속 중...
🟡  AC732 (단독항목) → 단독 저장
🟡  AC741 (단독항목) → 단독 저장
🟡  AC742 (단독항목) → 단독 저장
⚠️  8394 (AD201) 클릭했지만 하위 없음
🟡  AE100 (단독항목) → 단독 저장
🟡  AF100 (단독항목) → 단독 저장
🟡  AF200 (단독항목) → 단독 저장
🟡  AF300 (단독항목) → 단독 저장
🟡  AF400 (단독항목) → 단독 저장

▶ 페이지 10 접속 중...
❌ 8491 처리 실패 → Alert Text: 계속하시겠습니까?
Message: unexpected alert open: {Alert text

In [37]:
import os
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, UnexpectedAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 경로 설정
BASE_URL = "https://www.koicd.kr/ins/act.do"
BASE_DIR = os.path.abspath("suga_results")
JSON_DIR = os.path.join(BASE_DIR, "json_pages")
FAILED_FILE = os.path.join(BASE_DIR, "failed_codes.txt")
CSV_FILE = os.path.join(BASE_DIR, "suga_info_all.csv")

# 폴더 생성
os.makedirs(JSON_DIR, exist_ok=True)

# 드라이버 준비
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get(BASE_URL)
time.sleep(2)

all_data = []
failed_codes = []
page_num = 1

while True:
    print(f"\n▶ 페이지 {page_num} 접속 중...")

    try:
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.act_table")))
        rows = table.find_elements(By.CSS_SELECTOR, "tbody > tr")
    except Exception as e:
        print(f"❌ 테이블 로딩 실패: {e}")
        break

    page_data = []

    for row in rows:
        cls = row.get_attribute("class")
        if not cls or not cls.isdigit():
            continue

        try:
            suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
            kor_name = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
            eng_name = row.find_element(By.CSS_SELECTOR, "td:nth-child(4)").text.strip()

            try:
                toggle_btn = row.find_element(By.CSS_SELECTOR, "td.child b")
                driver.execute_script("arguments[0].click();", toggle_btn)
                time.sleep(0.5)

                sub_rows = table.find_elements(By.CSS_SELECTOR, f"tr.child{suga_code}.table")
                sub_data = []
                for sub in sub_rows:
                    tds = sub.find_elements(By.TAG_NAME, "td")
                    if len(tds) >= 3:
                        sub_code = tds[1].text.strip()
                        sub_name = tds[2].text.strip()
                        sub_data.append({
                            "parent_code": suga_code,
                            "sub_code": sub_code,
                            "sub_name": sub_name
                        })

                if sub_data:
                    page_data.extend(sub_data)
                else:
                    print(f"⚠️  {cls} ({suga_code}) 클릭했지만 하위 없음")
                    page_data.append({
                        "parent_code": suga_code,
                        "sub_code": "",
                        "sub_name": kor_name
                    })

            except NoSuchElementException:
                print(f"🟡  {suga_code} (단독항목) → 단독 저장")
                page_data.append({
                    "parent_code": suga_code,
                    "sub_code": "",
                    "sub_name": kor_name
                })

        except UnexpectedAlertPresentException as alert_ex:
            print(f"❌ {cls} ({suga_code}) 처리 실패 → Alert: {alert_ex}")
            failed_codes.append(suga_code)
            driver.switch_to.alert.dismiss()
            continue

        except Exception as e:
            print(f"❌ {cls} 처리 실패 → {e}")
            try:
                suga_code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                failed_codes.append(suga_code)
            except:
                continue

    # 페이지별 JSON 저장
    json_path = os.path.join(JSON_DIR, f"page_{page_num}.json")
    with open(json_path, "w", encoding="utf-8") as jf:
        json.dump(page_data, jf, ensure_ascii=False, indent=2)

    all_data.extend(page_data)

    # ✅ 중간 CSV 저장
    try:
        with open(CSV_FILE, "w", newline="", encoding="utf-8-sig") as cf:
            writer = csv.writer(cf)
            writer.writerow(["parent_code", "sub_code", "sub_name"])
            for row in all_data:
                writer.writerow([row["parent_code"], row["sub_code"], row["sub_name"]])
        if os.path.exists(CSV_FILE):
            print(f"✅ 중간 CSV 저장 완료: {CSV_FILE}")
    except Exception as e:
        print(f"❌ 중간 CSV 저장 실패: {e}")

    # 다음 페이지로 이동
    try:
        next_btn = driver.find_element(By.LINK_TEXT, str(page_num + 1))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
        time.sleep(0.5)
        next_btn.click()
        time.sleep(1)
        page_num += 1
    except Exception:
        print(f"⛔ 페이지 {page_num+1} 이동 실패. 중단.")
        break

# 실패 로그 저장
with open(FAILED_FILE, "w", encoding="utf-8") as ff:
    for code in failed_codes:
        ff.write(code + "\n")

print("\n📦 전체 데이터 통합 완료.")
print("✅ 모든 항목 정상 처리 완료.")

driver.quit()



▶ 페이지 1 접속 중...
🟡  AA222 (단독항목) → 단독 저장
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv

▶ 페이지 2 접속 중...
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv

▶ 페이지 3 접속 중...
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv

▶ 페이지 4 접속 중...
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv

▶ 페이지 5 접속 중...
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv

▶ 페이지 6 접속 중...
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv

▶ 페이지 7 접속 중...
🟡  AC001 (단독항목) → 단독 저장
🟡  AC002 (단독항목) → 단독 저장
🟡  AC003 (단독항목) → 단독 저장
🟡  AC004 (단독항목) → 단독 저장
🟡  AC005 (단독항목) → 단독 저장
🟡  AC006 (단독항목) → 단독 저장
🟡  AC007 (단독항목) → 단독 저장
🟡  AC031 (단독항목) → 단독 저장
🟡  AC032 (단독항목) → 단독 저장
🟡  AC033 (단독항목) → 단독 저장
✅ 중간 CSV 저장 완료: c:\Jimin\cg_SURI-Ag

In [38]:
import pandas as pd

newdf = pd.read_csv(r"C:\Jimin\cg_SURI-Agent\data_pipeline\bf4_selenium\suga_results\suga_info_all.csv")

newdf.info

<bound method DataFrame.info of      parent_code     sub_code                      sub_name
0          AA100  ―  AA100954        초진진찰료-치과의원, 보건의료원 내 치과
1          AA100  ―  AA100953        초진진찰료-치과의원, 보건의료원 내 치과
2          AA100  ―  AA100950        초진진찰료-치과의원, 보건의료원 내 치과
3          AA100  ―  AA100934        초진진찰료-치과의원, 보건의료원 내 치과
4          AA100  ―  AA100933        초진진찰료-치과의원, 보건의료원 내 치과
...          ...          ...                           ...
2885       AZ320  ―  AZ320600               장루/요루 교육·상담료_병원
2886       AZ320  ―  AZ320002               장루/요루 교육·상담료_병원
2887       AZ330  ―  AZ330600  만성신부전 교육·상담료-투석이 필요 없는 환자_병원
2888       AZ331  ―  AZ331600          만성신부전 교육·상담료-복막투석_병원
2889       AZ332  ―  AZ332600          만성신부전 교육·상담료-혈액투석_병원

[2890 rows x 3 columns]>

In [41]:
newdf.to_excel("수가코드목록_1.xlsx")